## Other resources you might find useful:

* [💡 how to process DICOM images to PNGs](https://www.kaggle.com/code/radek1/how-to-process-dicom-images-to-pngs)
* [📊 EDA + training a fast.ai model + submission 🚀](https://www.kaggle.com/code/radek1/eda-training-a-fast-ai-model-submission)

### Versions

* **v10**: res18, 4 epochs trained in submission, 256px, 3 splits, `CV: 0.046/0.076 LB: 0.05`, unfortunately had a bug in best thresh calulation
* **v11**: res18, 4 epochs trained in submission, 256px, 4 splits, ⏰ times out
* **v15**: res18, 4 epochs trained externally, 256px, CPU (the processing of images might be the most expensive part of inference! especially with resnet18, 2 CPUs -> 4 CPUs might be the way to go)
* **v16**: res18, 4 epochs trained externally, 256px, CPU, higher thresh (the means of predictions should be better than predictions from an individual model, a higher threshold *might* work better)
* **v17**: res18, like `v15`, but trained with cherry picking, either performing well in general (doubt it) or overfitted to local CV
* **v18**: tf_efficientnetv2_s, 512x512, single model
* **v19**: tf_efficientnetv2_s, 512x512, ensemble of 4 models
* **v20**: tf_efficientnetv2_s, like v19, but running with different pretrained weights, main difference -- this one runs on the CPU
* **v22**: like v18, but taking max instead of mean of predictions on images, that makes more sense but who knows if it will work better in practice
* **v23**: moving to new arch and adding VOI LUT transformation (might make the pipeline time out as it makes the processing a bit longer)
* **v24**: like v23 but with mean of individual thresholds recorded during training
* **v25**: like v23 but without VOI LUT processing
* **v26**: a single model with particularly good optimized pfbeta1 of just under 0.196
* **v27**: a single model with an optimized pfbeta1 of 0.212
* **v28**: a single model with an optimized pfbeta1 of 0.249
* **v30**: like v28, but with mean aggregation of predictions by `prediction_id`
* **v31**: a single model with an optimized pfbeta1 of 0.252
* **v32**: a single model with an optimized pfbeta1 of 0.207 trained on 1024x1024
* **v33**: similar to v32, but trained with augmentation and results in much higher threshold (plus locally the difference between pfbeta and optimized pfbeta is much greater)
* **v34**: like v32, but trained for one more epoch and on a different fold
* **v35**: an ensemble of v32 and v34

### Updates (account of working on this -- documents the despair of an ML practitioner we so often encounter!)

**12/04**: I find myself in the spot so often encountered by DL practitioners -- the model simply doesn't train! Or rather, it trains poorly to the point where one cannot be certain to what extent it is making good use of the available information.

This is an interesting case as a lot can be happening. From what I observed on the forums (special thanks go to Theo Viel and his awesome notebook: [RSNA Breast Baseline - Inference](https://www.kaggle.com/code/theoviel/rsna-breast-baseline-inference), now there is also [this notebook](https://www.kaggle.com/code/hengck23/notebooke04a738685) by hengck23), people are feeding the data to their model in a way that I wouldn't expect. Theo is only normalizing it to between 0 and 1.

But this might mean that some important information to the model is lost in the processing that I do here. Or it could be a hunderd of other small things the community has arrived at through so many RSNA competitions 🙂.

Three things to do:

* explore various ways of phrsing the problem within `fastai` (training with Sigmoid, Softmax, etc)
* get the pipeline on Kaggle working with what I have
    * to always have a working pipeline version (otherwise it is very easy to lose momentum on a project)
    * to be verifying if local improvements generalize to LB (so far, I have a very weak handle on what is going on in this competition, the more data I collect the better)
* after the above, see what happens if I feed data to my model either how Theo or hengck23 does it (normalized to [0,1], zero-centered and standard dev normalized)

**12/05**: I stayed up for more than half the night on Saturday and Sunday to work on this as I knew I wouldn't get much time to give this over the week. There can be an all-consuming quality to DL projects if you are not careful.

I reimplemented the entire pipeline in pure PyTorch... and it doesn't train in the SAME WAY as my fast.ai pipeline...

```
step:     1    val_loss: 0.686    pf1: 0.042    auc: 0.464
step:   500    val_loss: 0.108    pf1: 0.023    auc: 0.479
step:  1000    val_loss: 0.106    pf1: 0.023    auc: 0.579
step:  1500    val_loss: 0.105    pf1: 0.021    auc: 0.558
step:  2000    val_loss: 0.107    pf1: 0.020    auc: 0.524
step:  2500    val_loss: 0.106    pf1: 0.024    auc: 0.599
step:  3000    val_loss: 0.105    pf1: 0.023    auc: 0.575
step:  3500    val_loss: 0.104    pf1: 0.025    auc: 0.610
step:  4000    val_loss: 0.103    pf1: 0.027    auc: 0.623
step:  4500    val_loss: 0.104    pf1: 0.027    auc: 0.612
step:  5000    val_loss: 0.104    pf1: 0.027    auc: 0.607
step:  5125    val_loss: 0.104    pf1: 0.027    auc: 0.606
CPU times: user 12min 15s, sys: 38 s, total: 12min 53s
Wall time: 12min 56s
```

I replaced the training loop, how data is read, the data I am reading, added a metric from `torchmetrics` for another bit of sanity check and also did a couple of things that don't make sense but you essentially question everything when stuff doesn't train. The things I tried that didn't make much sense was training on larger images (why 512px should work if 256px doesn't train reasonably?) and training with sigmoid and BCE vs softmax and nll (with two classes they should be equivalent). Oh yeah, I also tried different models of course, directly from fast.ai, the one from Theo, pretrained, not pretrained, etc.

AND YET THERE ARE PEOPLE ON THE LB whose models do seem to train, potentially with ease.

What is the secret? What am I missing?

The only low hanging fruit I can think of now is addressing the class imbalance... Counting by images, there are roughly 2% of positive examples in the dataset. Maybe this can make a difference.

Other than that, the only 3 components I haven't tried replacing so far are:
* the researcher working on this (me)
* the splitting functionality into train and val
* my computer
* *maybe* the dataset, as in, would my code train on cats and dogs?

**12/05**: I think I found a way to train the model so that it works 😭😭😭😭😭 Don't want to jinx it though 🙂 Let me move it over to the Kaggle pipeline and let us see how it fares here 🙂 Writing [the thoughts down in this thread](https://www.kaggle.com/competitions/rsna-breast-cancer-detection/discussion/370587) was really helpful 🙏

<a id="section-one"></a>
# Training a fast.ai model


In [1]:
!unzip -q ../input/timm-with-dependencies/timm_all -d timm-with-dependencies
!pip install --no-index --find-links timm-with-dependencies timm
!pip install /kaggle/input/dicomsdl-offline-installer/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

from fastai.vision.learner import *
from fastai.data.all import *
from fastai.vision.all import *
from fastai.metrics import ActivationType

from sklearn.model_selection import StratifiedKFold
from collections import defaultdict
import pandas as pd
import numpy as np
from pdb import set_trace

Looking in links: timm-with-dependencies
Processing ./timm-with-dependencies/timm-0.6.12-py3-none-any.whl
Processing /kaggle/input/dicomsdl-offline-installer/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl


In [2]:
NUM_EPOCHS = 4
NUM_SPLITS = 4

RESIZE_TO = (1024, 1024)

DATA_PATH = '/kaggle/input/rsna-breast-cancer-detection'
TRAIN_IMAGE_DIR = '/kaggle/input/rsna-mammography-images-as-pngs/images_as_pngs_cv2_512'
TEST_DICOM_DIR = '/kaggle/input/rsna-breast-cancer-detection/test_images'
MODEL_PATH = '/kaggle/input/rsna-trained-model-weights/tf_effv2_s_208_402/tf_effv2_s_208_402'

label_smoothing_weights = torch.tensor([1,10]).float()
if torch.cuda.is_available():
    label_smoothing_weights = label_smoothing_weights.cuda()

## Creating stratified splits for training

In [3]:
train_csv = pd.read_csv(f'{DATA_PATH}/train.csv')
patient_id_any_cancer = train_csv.groupby('patient_id').cancer.max().reset_index()
skf = StratifiedKFold(NUM_SPLITS, shuffle=True, random_state=42)
splits = list(skf.split(patient_id_any_cancer.patient_id, patient_id_any_cancer.cancer))

## Defining some helper functions

I am defining some functionality here to make our life easier and to get us the last 10% of the way to a really good result.

Generally, none of this code is core to training or predicting, we could skip most of it and still be able to get a well trained model.


But here we want to push the boundaries of performance so let's get these things in 🙂

In [4]:
#https://www.kaggle.com/competitions/rsna-breast-cancer-detection/discussion/369267  
def pfbeta_torch(preds, labels, beta=1):
    if preds.dim() != 2 or (preds.dim() == 2 and preds.shape[1] !=2): raise ValueError('Houston, we got a problem')
    preds = preds[:, 1]
    preds = preds.clip(0, 1)
    y_true_count = labels.sum()
    ctp = preds[labels==1].sum()
    cfp = preds[labels==0].sum()
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0.0

# https://www.kaggle.com/competitions/rsna-breast-cancer-detection/discussion/369886    
def pfbeta_torch_thresh(preds, labels):
    optimized_preds = optimize_preds(preds, labels)
    return pfbeta_torch(optimized_preds, labels)

def optimize_preds(preds, labels=None, thresh=None, return_thresh=False, print_results=False):
    preds = preds.clone()
    if labels is not None: without_thresh = pfbeta_torch(preds, labels)
    
    if not thresh and labels is not None:
        threshs = np.linspace(0, 1, 101)
        f1s = [pfbeta_torch((preds > thr).float(), labels) for thr in threshs]
        idx = np.argmax(f1s)
        thresh, best_pfbeta = threshs[idx], f1s[idx]

    preds = (preds > thresh).float()

    if print_results:
        print(f'without optimization: {without_thresh}')
        pfbeta = pfbeta_torch(preds, labels)
        print(f'with optimization: {pfbeta}')
        print(f'best_thresh = {thresh}')
    if return_thresh:
        return thresh
    return preds

fn2label = {fn: cancer_or_not for fn, cancer_or_not in zip(train_csv['image_id'].astype('str'), train_csv['cancer'])}

def splitting_func(paths):
    train = []
    valid = []
    for idx, path in enumerate(paths):
        if int(path.parent.name) in patient_id_any_cancer.iloc[splits[SPLIT][0]].patient_id.values:
            train.append(idx)
        else:
            valid.append(idx)
    return train, valid

def label_func(path):
    return fn2label[path.stem]

def get_items(image_dir_path):
    items = []
    for p in get_image_files(image_dir_path):
        items.append(p)
        if p.stem in fn2label and int(p.parent.name) in patient_id_any_cancer.iloc[splits[SPLIT][0]].patient_id.values:
            if label_func(p) == 1:
                for _ in range(5):
                    items.append(p)
    return items

Wrapping getting data and getting a model into functions -- this way our logic for training will be cleaner to read.

In [5]:
from timm.models.layers.adaptive_avgmax_pool import SelectAdaptivePool2d
from torch.nn import Flatten

def get_dataloaders():
    train_image_path = TRAIN_IMAGE_DIR

    dblock = DataBlock(
        blocks    = (ImageBlock, CategoryBlock),
        get_items = get_items,
        get_y = label_func,
        splitter  = splitting_func,
        batch_tfms=[Flip()],
    )
    dsets = dblock.datasets(train_image_path)
    return dblock.dataloaders(train_image_path, batch_size=32)

def get_learner(arch=resnet18):
    learner = vision_learner(
        get_dataloaders(),
        arch,
        custom_head=nn.Sequential(SelectAdaptivePool2d(pool_type='avg', flatten=Flatten()), nn.Linear(1280, 2)),
        metrics=[
            error_rate,
            AccumMetric(pfbeta_torch, activation=ActivationType.Softmax, flatten=False),
            AccumMetric(pfbeta_torch_thresh, activation=ActivationType.Softmax, flatten=False)
        ],
        loss_func=CrossEntropyLossFlat(weight=torch.tensor([1,50]).float()),
        pretrained=True,
        normalize=False
    ).to_fp16()
    return learner

## Creating the learner and training

In [6]:
# This is a dependency that is needed for reading DICOM images

try:
    import pylibjpeg
except:
    !rm -rf /root/.cache/torch/hub/checkpoints/
    !mkdir -p /root/.cache/torch/hub/checkpoints/
    !pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
    !pip install /kaggle/input/rsna-2022-whl/{torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl,torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl}

# copying the pretrained weights

if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '/kaggle/input/pretrained-model-weights-for-fastai/resnet18-f37072fd.pth' '/root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth'
!cp '/kaggle/input/pretrained-model-weights-for-fastai/tf_efficientnetv2_s-eb54923e.pth' '/root/.cache/torch/hub/checkpoints/tf_efficientnetv2_s-eb54923e.pth'

Processing /kaggle/input/rsna-2022-whl/pydicom-2.3.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
pydicom is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/rsna-2022-whl/torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /kaggle/input/rsna-2022-whl/torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0


In [7]:
%%time

preds, labels = [], []

SPLIT = 0 # our learner needs this to construct its dataloaders...
learn = get_learner('tf_efficientnetv2_s')

# instead of training, to conserve pipeline time, I am uploading models trained locally
# uncomment the lines below for training
  
# for SPLIT in range(NUM_SPLITS):
#     learn = get_learner()
#     learn.unfreeze()
#     learn.fit_one_cycle(NUM_EPOCHS, 1e-4, pct_start=0.1)
#     learn.save(f'{MODEL_PATH}/{SPLIT}')
        
#     output = learn.get_preds()
#     preds.append(output[0])
#     labels.append(output[1])

CPU times: user 1min 26s, sys: 1.94 s, total: 1min 28s
Wall time: 2min 28s


In [8]:
# threshold = optimize_preds(torch.cat(preds), torch.cat(labels), return_thresh=True, print_results=True)
threshold = 0.402

# Predicting on test<a id="section-two">

In [9]:
# import pydicom
# from pydicom.pixel_data_handlers.util import apply_voi_lut
import dicomsdl
    
from pathlib import Path
import multiprocessing as mp
import cv2

!rm -rf test_resized_{RESIZE_TO[0]}

def dicom_file_to_ary(path):
    dcm_file = dicomsdl.open(str(path))
    data = dcm_file.pixelData()

    data = (data - data.min()) / (data.max() - data.min())

    if dcm_file.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
        data = 1 - data

    data = cv2.resize(data, RESIZE_TO)
    data = (data * 255).astype(np.uint8)
    return data

directories = list(Path(TEST_DICOM_DIR).iterdir())

def process_directory(directory_path):
    parent_directory = str(directory_path).split('/')[-1]
    !mkdir -p test_resized_{RESIZE_TO[0]}/{parent_directory}
    for image_path in directory_path.iterdir():
        processed_ary = dicom_file_to_ary(image_path)
        cv2.imwrite(
            f'test_resized_{RESIZE_TO[0]}/{parent_directory}/{image_path.stem}.png',
            processed_ary
        )

with mp.Pool(mp.cpu_count()) as p:
    p.map(process_directory, directories)

In [10]:
%%time

preds_all = []

test_dl = learn.dls.test_dl(get_image_files(f'test_resized_{RESIZE_TO[0]}'))
for SPLIT in range(NUM_SPLITS):
    learn.load(f'{MODEL_PATH}/{SPLIT}')
    preds, _ = learn.get_preds(dl=test_dl)
    preds_all.append(preds)

CPU times: user 3.7 s, sys: 1.9 s, total: 5.6 s
Wall time: 19.7 s


In [11]:
preds = torch.zeros_like(preds_all[0])
for pred in preds_all:
    preds += pred

preds /= NUM_SPLITS


preds = optimize_preds(preds, thresh=threshold)
image_ids = [path.stem for path in test_dl.items]

image_id2pred = defaultdict(lambda: 0)
for image_id, pred in zip(image_ids, preds[:, 1]):
    image_id2pred[int(image_id)] = pred.item()

<a id="section-three"></a>
# Making a submission

In [12]:
test_csv = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')

prediction_ids = []
preds = []

for _, row in test_csv.iterrows():
    prediction_ids.append(row.prediction_id)
    preds.append(image_id2pred[row.image_id])

submission = pd.DataFrame(data={'prediction_id': prediction_ids, 'cancer': preds}).groupby('prediction_id').max().reset_index()
submission.head()

,prediction_id,cancer
0,10008_L,0.0
1,10008_R,0.0


In [13]:
submission.to_csv('submission.csv', index=False)

And that's it! Thank you very much for reading! 🙂

**If you enjoyed the notebook, please upvote! 🙏 Thank you, appreciate your support!**

Happy Kaggling 🥳
